In [1]:
import pandas as pd
import openmatrix as omx
import numpy as np
import os

# Station Info

In [2]:
station_obs_names_df = pd.read_csv(r"E:\GitHub\Resources\1-TDM\Station-Lookups\station-obs-names.csv")
crt_station_obs_names_df = station_obs_names_df[station_obs_names_df['mode']=='CRT']
crt_station_obs_names_df = crt_station_obs_names_df.groupby(['LAT','LON'],as_index=False).agg(STATION=('STATION','first'),TAZID=('TAZID','max'))

# Sort by LAT (descending) and LON (ascending) for north-to-south ordering
unique_stations_df = crt_station_obs_names_df.drop_duplicates(subset=['STATION']).sort_values(
    by=['LAT', 'LON'], 
    ascending=[False, False]
).reset_index(drop=True)

# Assign row numbers only to unique STATIONs
unique_stations_df['RowNumber'] = range(1, len(unique_stations_df) + 1)

# Merge back to original DataFrame to retain all rows
crt_station_obs_names_df = crt_station_obs_names_df.merge(
    unique_stations_df[['STATION', 'RowNumber']], 
    on='STATION', 
    how='left'
)

crt_station_obs_names_df['STATION_ID'] = crt_station_obs_names_df['RowNumber'].apply(lambda x: f"{x:02d}") + '-' + crt_station_obs_names_df['STATION']

# USE WALK INSTEAD______
## CLOSE STATION FIXES
## CHANGE ALL NORTH TEMPLE STATION TAZs to 933 since TAZ station is in actually would drive to SL Central
## CHANGE ALL SALT LAKE CENTRAL STATION TAZs to 1023 since TAZ station is acutally in this one
#crt_station_obs_names_df.loc[crt_station_obs_names_df['STATION']=='North Temple', 'TAZID'] = 933
#crt_station_obs_names_df.loc[crt_station_obs_names_df['STATION']=='Salt Lake Central', 'TAZID'] = 1023

crt_station_obs_names_df

,LAT,LON,STATION,TAZID,RowNumber,STATION_ID
0,40.225440,-111.660632,Provo Central,3001.0,15,15-Provo Central
1,40.280140,-111.725489,Orem Central,2882.0,14,14-Orem Central
2,40.374774,-111.820649,American Fork,2728.0,13,13-American Fork
3,40.425196,-111.896354,Lehi,2605.0,12,12-Lehi
4,40.515484,-111.904407,Draper,2117.0,11,11-Draper
5,40.563155,-111.900753,South Jordan,1973.0,10,10-South Jordan
6,40.659631,-111.895661,Murray Central,1633.0,9,09-Murray Central
7,40.659758,-111.896432,Murray Central,1633.0,9,09-Murray Central
8,40.660269,-111.895282,Murray Central,1633.0,9,09-Murray Central
9,40.761721,-111.908301,Salt Lake Central,1024.0,8,08-Salt Lake Central


In [3]:
# Perform Cartesian Join (self-join)
crossed_df = crt_station_obs_names_df.merge(
    crt_station_obs_names_df, 
    how='cross', 
    suffixes=('_1', '_2')
)

# Filter where STATION_ID_1 < STATION_ID_2 alphabetically
crt_station_obs_names_crossed_df = crossed_df[crossed_df['STATION_ID_1'] < crossed_df['STATION_ID_2']]
crt_station_obs_names_crossed_df = crt_station_obs_names_crossed_df[['STATION_ID_1','STATION_ID_2','TAZID_1','TAZID_2']]

crt_station_obs_names_crossed_df

,STATION_ID_1,STATION_ID_2,TAZID_1,TAZID_2
19,14-Orem Central,15-Provo Central,2882.0,3001.0
38,13-American Fork,15-Provo Central,2728.0,3001.0
39,13-American Fork,14-Orem Central,2728.0,2882.0
57,12-Lehi,15-Provo Central,2605.0,3001.0
58,12-Lehi,14-Orem Central,2605.0,2882.0
...,...,...,...,...
355,01-Ogden,06-Woods Cross,368.0,863.0
356,01-Ogden,05-Farmington,368.0,780.0
357,01-Ogden,04-Layton,368.0,737.0
358,01-Ogden,03-Clearfield,368.0,663.0


# TDM Prep

In [4]:
model_paths = {
#   "E2.14.1-TDM-Recalib"                            : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.14.1/BY_2019",
#   "E2.14.2-TDM-Recalib-100-divisor-removed"        : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.14.2/BY_2019",
#   "E2.14.3-TDM-Recalib-second-part-ivt-removed"    : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.14.3/BY_2019",
    "E2.14.4-TDM-Recalib-IvtClean-RunFac2.5"         : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.14.4/BY_2019",
    "E2.7.3-IVT-Test"                                : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.7.3/BY_2019",
#   "E2.13.2-Single-TLF-RunFac-2.5"                  : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.13.2/BY_2019",
#   "E2.13.3-Purpose-TLF-RunFac-2.5"                 : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.13.3/BY_2019",
#   "E2.13.4-PurpPrd-TLF-RunFac-2.5"                 : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.13.4/BY_2019",
#   "E2.13.4.2-PurpPrd-TLF-RunFac-2.5-NonZero-Calib" : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.13.4.2/BY_2019",
    "E2.13.4.3-PurpPrd-TLF-RunFac-2.5"               : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.13.4.3/BY_2019",
#   "E2.13.5-PurpPrd-TLF-RunFac-2.5-wIvtTest"        : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.13.5/BY_2019",
    "E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest"      : "E:/GitHub/WF-TDM-v9x/Scenarios/v920-E2.13.5.2/BY_2019"
}

In [5]:
# Define transit skim files
transit_skim_loc = [
    r"1a_Skims\skm_d8_Pk.omx", r"1a_Skims\skm_d8_Ok.omx",
    r"1a_Skims\skm_w8_Pk.omx", r"1a_Skims\skm_w8_Ok.omx"
]

# Define invalid TAZs
invalid_model_taz_list = np.arange(3547, 3601)

# Define purpose, periods, and access modes
purposes = ['HBW', 'HBO', 'NHB', 'HBC']
periods = ['PK', 'OK']
accesses = ['dCRT', 'wCRT']

skimloc_mapping = {
    ('PK', 'dCRT'): 0,
    ('PK', 'wCRT'): 2,
    ('OK', 'dCRT'): 1,
    ('OK', 'wCRT'): 3
}
access_mapping = {'dCRT': 'Drive', 'wCRT': 'Walk'}

# Function to convert model data into DataFrame
def create_model_mtx_to_df(trips_file_name, crdist_file_name, trips_mtx_name, crdist_mtx_name='D8', delZero=True):
    trips_file = omx.open_file(trips_file_name)
    trips_mtx = np.array(trips_file[trips_mtx_name])
    crdist_file = omx.open_file(crdist_file_name)
    crdist_mtx = np.array(crdist_file[crdist_mtx_name])

    trips_df = pd.DataFrame(pd.DataFrame(trips_mtx).stack()).rename({0: 'trips_count'}, axis=1)
    crdist_df = pd.DataFrame(pd.DataFrame(crdist_mtx).stack()).rename({0: 'cr_travel_distance'}, axis=1)

    model_df = pd.concat([trips_df, crdist_df], axis=1).reset_index().rename(
        {'level_0': 'p_TAZID', 'level_1': 'a_TAZID'}, axis=1)

    model_df['p_TAZID'] += 1
    model_df['a_TAZID'] += 1
    model_df = model_df[~model_df['p_TAZID'].isin(invalid_model_taz_list) & ~model_df['a_TAZID'].isin(invalid_model_taz_list)]

    # **Filter out rows where trips_count == 0**
    if delZero: 
        model_df = model_df[model_df['trips_count'] > 0]

    return model_df

# Dictionary to store results from all models
model_results = {}

# Iterate through model versions
for version, Model_path in model_paths.items():
    print("")
    print(f"Processing Model Version: {version}")
    model_data_loc = os.path.join(Model_path, r"4_ModeChoice\1a_Skims")
    model_data_loc2 = os.path.join(Model_path, r"4_ModeChoice")

    for purpose in purposes:
        print(f"...{purpose}", end="")
        for period in periods:
            print(f" {period}", end="")
            if purpose=='HBC' and period=='OK':
                print(f" skip", end="")
                continue
            for access in accesses:
                print(f" {access}", end="")

                access_mapped = access_mapping.get(access, 'Unknown')  # Returns 'Drive'

                model_df = create_model_mtx_to_df(
                    trips_file_name=os.path.join(model_data_loc2, '2_DetailedTripMatrices', f"{purpose}_trips_allsegs_{period}.omx"),
                    crdist_file_name=os.path.join(model_data_loc2, transit_skim_loc[skimloc_mapping.get((period, access), None)]),
                    trips_mtx_name=access,
                    delZero = True
                )

                model_df['trips_count'] /= 100
                model_df = model_df[model_df['cr_travel_distance'] > 0]

                key = f"{purpose}_{access}_{period}_{version}"
                model_results[key] = model_df.copy()

print("")
print("Done!")


Processing Model Version: E2.14.4-TDM-Recalib-IvtClean-RunFac2.5
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Processing Model Version: E2.7.3-IVT-Test
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Processing Model Version: E2.13.4.3-PurpPrd-TLF-RunFac-2.5
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Processing Model Version: E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Done!


In [6]:
model_results_df = pd.concat(model_results).reset_index().drop(columns=['level_1'])

# Split 'level_0' into four parts based on the first three underscores
split_cols = model_results_df['level_0'].str.split('_', n=3, expand=True)

model_results_df.drop(columns=['level_0'], inplace=True)

# Assign the first three parts to new columns
model_results_df['purpose'] = split_cols[0]
model_results_df['mode'] = split_cols[1]
model_results_df['period'] = split_cols[2]
model_results_df['model'] = split_cols[3]

model_results_df

,p_TAZID,a_TAZID,trips_count,cr_travel_distance,purpose,mode,period,model
0,3,634,0.0001,11.24,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5
1,3,659,0.0001,11.24,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5
2,3,663,0.0002,11.24,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5
3,3,720,0.0002,14.92,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5
4,3,780,0.0005,20.80,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5
...,...,...,...,...,...,...,...,...
18361996,3180,2848,0.0021,5.24,HBC,wCRT,PK,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest
18361997,3195,2848,0.0330,5.24,HBC,wCRT,PK,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest
18361998,3210,2848,0.1438,5.24,HBC,wCRT,PK,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest
18361999,3211,2848,0.1011,5.24,HBC,wCRT,PK,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest


In [7]:
# Get station-to-station crt distance
# using TAZ of stations as proxy for station-to-station distance that can then be joined with crt distance matrix to parse out station-to-station pairs

crdist_file = omx.open_file(r"E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.13.5.2\BY_2019\4_ModeChoice\1a_Skims\skm_w8_Pk.omx")

crdist_df = pd.DataFrame(pd.DataFrame(np.array(crdist_file['D8'])).stack()).rename({0: 'cr_travel_distance'}, axis=1).reset_index().rename(columns={'level_0':'I','level_1':'J'})

# index start of omx is 0, voyager is 1
crdist_df['I'] += 1
crdist_df['J'] += 1

crdist_df = crdist_df[crdist_df['cr_travel_distance']>0]
crdist_df

,I,J,cr_travel_distance
269042,75,497,4.32
269045,75,500,4.32
269047,75,502,4.32
269209,75,664,11.24
269282,75,737,14.92
...,...,...,...
12566475,3463,2878,5.24
12566476,3463,2879,5.24
12566478,3463,2881,5.24
12566479,3463,2882,5.24


In [8]:
# Get station-to-station crt distance
# using TAZ of stations as proxy for station-to-station distance that can then be joined with crt distance matrix to parse out station-to-station pairs

model_results[key]

# use walk, since drive to crt gives interesting results between SLC Central and North Temple
combined_for_station_pairs_df = pd.merge(crdist_df, crt_station_obs_names_crossed_df, left_on=['I','J'], right_on=['TAZID_1','TAZID_2'])
station_pair_crt_distances_df = combined_for_station_pairs_df[['cr_travel_distance','STATION_ID_1','STATION_ID_2']].drop_duplicates()
station_pair_crt_distances_df

# manually add -- manually created matrix from station to next station distances to find closest
_df_temp = pd.DataFrame([
    [0.78, '07-North Temple', '08-Salt Lake Central' ],
    # THESE ARE STRANGE, IT DUPLICATES DISTANCE BETWEEN SOME STATION PAIRS!!!
    [8.64, '01-Ogden', '02-Roy'], # 4.32 x 2
    [14.32, '08-Salt Lake Central', '09-Murray Central'], # 7.16 x 2
    [6.48, '10-South Jordan', '11-Draper'] # 3.24 x 2
], columns=['cr_travel_distance', 'STATION_ID_1', 'STATION_ID_2'])
station_pair_crt_distances_df = pd.concat([station_pair_crt_distances_df, _df_temp])

#
## IMPORTANT NOTE: Clearfield to Salt Lake Central and Woods Cross to Draper have the same distance!
## SO JUST COMBINE THEM FOR EASE
#
# Group by cr_travel_distance and concatenate STATION_ID_1 and STATION_ID_2
station_pair_crt_distances_df = station_pair_crt_distances_df.groupby("cr_travel_distance").agg({
    "STATION_ID_1": lambda x: ", ".join(map(str, x)),
    "STATION_ID_2": lambda x: ", ".join(map(str, x))
}).reset_index()

station_pair_crt_distances_df.sort_values(
    by=["STATION_ID_1", "STATION_ID_2"], ascending=[True, True]
)


,cr_travel_distance,STATION_ID_1,STATION_ID_2
3,4.32,01-Ogden,02-Roy
16,8.64,01-Ogden,02-Roy
22,11.24,01-Ogden,03-Clearfield
30,14.92,01-Ogden,04-Layton
40,20.80,01-Ogden,05-Farmington
...,...,...,...
27,14.12,12-Lehi,14-Orem Central
39,19.36,12-Lehi,15-Provo Central
15,8.44,13-American Fork,14-Orem Central
25,13.68,13-American Fork,15-Provo Central


In [9]:
model_results_with_station_pairs_df = pd.merge(model_results_df, station_pair_crt_distances_df, on=['cr_travel_distance'], how='left')
model_results_with_station_pairs_df

,p_TAZID,a_TAZID,trips_count,cr_travel_distance,purpose,mode,period,model,STATION_ID_1,STATION_ID_2
0,3,634,0.0001,11.24,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5,01-Ogden,03-Clearfield
1,3,659,0.0001,11.24,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5,01-Ogden,03-Clearfield
2,3,663,0.0002,11.24,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5,01-Ogden,03-Clearfield
3,3,720,0.0002,14.92,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5,01-Ogden,04-Layton
4,3,780,0.0005,20.80,HBW,dCRT,PK,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5,01-Ogden,05-Farmington
...,...,...,...,...,...,...,...,...,...,...
18361996,3180,2848,0.0021,5.24,HBC,wCRT,PK,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest,14-Orem Central,15-Provo Central
18361997,3195,2848,0.0330,5.24,HBC,wCRT,PK,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest,14-Orem Central,15-Provo Central
18361998,3210,2848,0.1438,5.24,HBC,wCRT,PK,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest,14-Orem Central,15-Provo Central
18361999,3211,2848,0.1011,5.24,HBC,wCRT,PK,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest,14-Orem Central,15-Provo Central


** MAKE SURE ALL RECORDS HAVE STATIONS ASSIGNED **

In [10]:
model_results_with_station_pairs_df[
    model_results_with_station_pairs_df.isnull().any(axis=1)
]['cr_travel_distance'].dropna().unique()

array([], dtype=float64)

In [11]:
model_results_with_station_pairs_df.groupby(['model'], as_index=False).agg(trips_total=('trips_count','sum'))

,model,trips_total
0,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,19864.9922
1,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest,19949.0246
2,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5,19851.6567
3,E2.7.3-IVT-Test,19916.8142


In [12]:
model_results_with_station_pairs_df.loc[
    model_results_with_station_pairs_df['STATION_ID_1'].isna(), 'trips_count'
].sum()

0.0

In [13]:
# make sure records are the same number

# Check the number of records
count_1 = len(model_results_with_station_pairs_df)
count_2 = len(model_results_df)

# Print the result
if count_1 == count_2:
    print(f"✅ Both DataFrames have the same number of records: {count_1}")
else:
    print(f"❌ Mismatch: model_results_with_station_pairs_df has {count_1} records, "
          f"while model_results_df has {count_2} records.")


✅ Both DataFrames have the same number of records: 18362001


In [14]:
model_results_with_station_pairs_df.groupby(['model','mode','purpose','period'], as_index=False).agg(trips_total=('trips_count','sum'))

,model,mode,purpose,period,trips_total
0,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,dCRT,HBC,PK,3006.1162
1,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,dCRT,HBO,OK,1084.5083
2,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,dCRT,HBO,PK,1165.2092
3,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,dCRT,HBW,OK,1651.1368
4,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,dCRT,HBW,PK,6123.8199
5,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,dCRT,NHB,OK,236.4130
6,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,dCRT,NHB,PK,159.6804
7,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,wCRT,HBC,PK,807.7599
8,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,wCRT,HBO,OK,1451.5619
9,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,wCRT,HBO,PK,659.4326


In [15]:
model_results_CRT_df = model_results_with_station_pairs_df.groupby(['model','STATION_ID_1','STATION_ID_2','purpose','period','mode'], as_index=False).agg(trips_total=('trips_count','sum'))
model_results_CRT_df

,model,STATION_ID_1,STATION_ID_2,purpose,period,mode,trips_total
0,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBC,PK,dCRT,6.9294
1,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBC,PK,wCRT,7.2227
2,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,OK,dCRT,1.3716
3,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,OK,wCRT,47.8966
4,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,PK,dCRT,0.4969
...,...,...,...,...,...,...,...
5501,E2.7.3-IVT-Test,14-Orem Central,15-Provo Central,HBW,PK,wCRT,34.5157
5502,E2.7.3-IVT-Test,14-Orem Central,15-Provo Central,NHB,OK,dCRT,0.2692
5503,E2.7.3-IVT-Test,14-Orem Central,15-Provo Central,NHB,OK,wCRT,12.8376
5504,E2.7.3-IVT-Test,14-Orem Central,15-Provo Central,NHB,PK,dCRT,0.0687


In [25]:
model_results_CRT_df.groupby(['model'], as_index=False).agg(trips_total=('trips_total','sum'))

,model,trips_total
0,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,19864.9922
1,E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest,19949.0246
2,E2.14.4-TDM-Recalib-IvtClean-RunFac2.5,19851.6567
3,E2.7.3-IVT-Test,19916.8142


# OBS Prep

In [32]:
obs_df = pd.read_csv(r"C:\Users\bhereth\Documents\2019 Final Weighted UTA OD Data - 2022-04-05 - processed.csv", low_memory=False)
obs_crt_df = obs_df[(obs_df['Linked_Mode_txt']=='CRT')]

# all HBC are peak period
obs_crt_df.loc[obs_crt_df['Purp5_text'] == 'HBC', 'PK_OK'] = 'PK'

obs_crt_df

,id,date_completed,final_route_surveyed_code,final_route_surveyed,direction,mode,purpose,resident_or_visitor_code,home_or_hotel_address,home_or_hotel_city,...,FirstOn_lat,FirstOn_lon,LastOff_lat,LastOff_lon,p_Stop_lat,p_Stop_lon,a_Stop_lat,a_Stop_lon,unlinked_weight_adj,linked_weight_adj
3,30.0,9-Sep-19,SLC_1_703_00,TRAX RED LINE TO DAYBREAK PARKWAY STATION,0.0,1.0,2.0,1.0,262 E Whitlock Ave,South Salt Lake,...,40.724091,-111.896892,40.280140,-111.725489,40.724091,-111.896892,40.280140,-111.725489,11.440,5.720
5,32.0,9-Sep-19,SLC_1_701_00,TRAX BLUE LINE TO DRAPER TOWN CENTER STATION,0.0,1.0,1.0,1.0,10404 Avondale Dr,Cedar Hills,...,40.374774,-111.820649,40.767452,-111.891101,40.374774,-111.820649,40.767452,-111.891101,18.136,9.068
17,61.0,9-Sep-19,SLC_1_701_01,TRAX BLUE LINE TO SALT LAKE CENTRAL STATION,1.0,1.0,4.0,2.0,10333 S Jordan Gateway,South Jordan,...,40.563155,-111.900753,40.769344,-111.901146,40.563155,-111.900753,40.769344,-111.901146,3.265,1.633
46,212.0,11-Sep-19,SLC_1_701_01,TRAX BLUE LINE TO SALT LAKE CENTRAL STATION,1.0,1.0,1.0,1.0,W Coyote Cir,Saratoga Springs,...,40.425196,-111.896354,40.763855,-111.891071,40.425196,-111.896354,40.763855,-111.891071,9.312,4.656
58,236.0,11-Sep-19,SLC_1_704_01,TRAX GREEN LINE TO AIRPORT STATION,1.0,1.0,3.0,1.0,622 23rd St,Ogden,...,41.224285,-111.980631,40.783280,-111.980172,41.224285,-111.980631,40.783280,-111.980172,7.603,3.801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13403,22722.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,641 1850 N St,Ogden,...,40.763855,-111.891071,41.224285,-111.980631,41.224285,-111.980631,40.763855,-111.891071,11.951,5.976
13409,22737.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,185 E 1050 N,Bountiful,...,40.768217,-111.891081,40.880457,-111.903151,40.880457,-111.903151,40.768217,-111.891081,17.630,8.815
13410,22738.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,2803 W 1475 N,Layton,...,40.768217,-111.891081,41.094769,-112.013807,41.094769,-112.013807,40.768217,-111.891081,11.951,5.976
13414,22744.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,W 4800 S,Roy,...,40.772532,-111.905124,41.188757,-112.039378,41.188757,-112.039378,40.772532,-111.905124,11.238,11.238


In [33]:
# check total should be around 20,500, drive to CRT around 15,000)
obs_crt_df['linked_weight_adj'].sum()

20482.226000000002

In [34]:
stop_fields_lat_lon_df = pd.DataFrame([
    ['stop_on_lat'            , 'stop_on_long'              ],
    ['stop_off_lat'           , 'stop_off_long'             ],
    ['prev_tran_1_on_bus_lat' , 'prev_tran_1_on_bus_long'   ],
    ['prev_tran_1_off_bus_lat', 'prev_tran_1_off_bus_long'  ],
    ['prev_tran_2_on_bus_lat' , 'prev_tran_2_on_bus_long'   ],
    ['prev_tran_2_off_bus_lat', 'prev_tran_2_off_bus_long'  ],
    ['prev_tran_3_on_bus_lat' , 'prev_tran_3_on_bus_long'   ],
    ['prev_tran_3_off_bus_lat', 'prev_tran_3_off_bus_long'  ],
    ['prev_tran_4_on_bus_lat' , 'prev_tran_4_on_bus_long'   ],
    ['prev_tran_4_off_bus_lat', 'prev_tran_4_off_bus_long'  ],
    ['next_tran_1_on_bus_lat' , 'next_tran_1_on_bus_long'   ],
    ['next_tran_1_off_bus_lat', 'next_tran_1_off_bus_long'  ],
    ['next_tran_2_on_bus_lat' , 'next_tran_2_on_bus_long'   ],
    ['next_tran_2_off_bus_lat', 'next_tran_2_off_bus_long'  ],
    ['next_tran_3_on_bus_lat' , 'next_tran_3_on_bus_long'   ],
    ['next_tran_3_off_bus_lat', 'next_tran_3_off_bus_long'  ],
    ['next_tran_4_on_bus_lat' , 'next_tran_4_on_bus_long'   ],
    ['next_tran_4_off_bus_lat', 'next_tran_4_off_bus_long'  ]
], columns=['stop_lat','stop_lon'])
stop_fields_lat_lon_df

,stop_lat,stop_lon
0,stop_on_lat,stop_on_long
1,stop_off_lat,stop_off_long
2,prev_tran_1_on_bus_lat,prev_tran_1_on_bus_long
3,prev_tran_1_off_bus_lat,prev_tran_1_off_bus_long
4,prev_tran_2_on_bus_lat,prev_tran_2_on_bus_long
5,prev_tran_2_off_bus_lat,prev_tran_2_off_bus_long
6,prev_tran_3_on_bus_lat,prev_tran_3_on_bus_long
7,prev_tran_3_off_bus_lat,prev_tran_3_off_bus_long
8,prev_tran_4_on_bus_lat,prev_tran_4_on_bus_long
9,prev_tran_4_off_bus_lat,prev_tran_4_off_bus_long


In [35]:
obs_crt_df.columns.to_list()

['id',
 'date_completed',
 'final_route_surveyed_code',
 'final_route_surveyed',
 'direction',
 'mode',
 'purpose',
 'resident_or_visitor_code',
 'home_or_hotel_address',
 'home_or_hotel_city',
 'home_or_hotel_state',
 'home_or_hotel_zip',
 'home_or_hotel_lat',
 'home_or_hotel_long',
 'final_origin_place_type_code',
 'origin_address',
 'origin_city',
 'origin_state',
 'origin_zip',
 'origin_lat',
 'origin_long',
 'prev_transfers_code',
 'trip_first_route_code',
 'trip_first_route',
 'trip_first_route_other',
 'trip_second_route_code',
 'trip_second_route',
 'trip_second_route_other',
 'trip_third_route_code',
 'trip_third_route',
 'trip_third_route_other',
 'trip_fourth_route_code',
 'trip_fourth_route',
 'trip_fourth_route_other',
 'final_origin_transport_code',
 'final_destin_place_type_code',
 'destin_address',
 'destin_city',
 'destin_state',
 'destin_zip',
 'destin_lat',
 'destin_long',
 'next_transfers_code',
 'trip_next_route_code',
 'trip_next_route',
 'trip_next_route_other',


In [36]:
filtered_obs_fields = ['id','Purp5_text','PK_OK','Ac_Mode_Model','Linked_Mode_txt','linked_weight_adj']
filtered_obs_fields

['id',
 'Purp5_text',
 'PK_OK',
 'Ac_Mode_Model',
 'Linked_Mode_txt',
 'linked_weight_adj']

In [37]:
obs_crt_stations_df = pd.DataFrame()

for _, row in stop_fields_lat_lon_df.iterrows():
    #print(_)

    # get station name
    _df = pd.merge(obs_crt_df, crt_station_obs_names_df, left_on=[row['stop_lat'], row['stop_lon']], right_on=['LAT','LON'])

    _df = _df[filtered_obs_fields + ['STATION_ID']]

    obs_crt_stations_df = pd.concat([obs_crt_stations_df, _df])

display(obs_crt_stations_df)

,id,Purp5_text,PK_OK,Ac_Mode_Model,Linked_Mode_txt,linked_weight_adj,STATION_ID
0,32.0,HBW,PK,Drive,CRT,9.068,08-Salt Lake Central
1,61.0,NHB,OK,Walk,CRT,1.633,09-Murray Central
2,338.0,HBC,PK,Drive,CRT,1.697,09-Murray Central
3,430.0,HBO,OK,Walk,CRT,1.573,07-North Temple
4,434.0,HBW,OK,Drive,CRT,2.360,07-North Temple
...,...,...,...,...,...,...,...
29,18561.0,HBW,OK,Walk,CRT,6.300,11-Draper
30,19897.0,HBW,PK,Drive,CRT,5.046,02-Roy
31,21157.0,NHB,PK,Walk,CRT,7.784,05-Farmington
0,11872.0,HBW,OK,Drive,CRT,2.589,07-North Temple


In [38]:
obs_crt_stations_unique_df = obs_crt_stations_df.drop_duplicates()

grouped_df = obs_crt_stations_unique_df.groupby(
    ['id', 'linked_weight_adj', 'Purp5_text', 'PK_OK', 'Ac_Mode_Model', 'Linked_Mode_txt'],
    as_index=False
).agg({'STATION_ID': lambda x: sorted(x)})

# Extract STATION_ID_1 and STATION_ID_2 from the list
grouped_df['STATION_ID_1'] = grouped_df['STATION_ID'].apply(lambda x: x[0] if len(x) > 0 else None)
grouped_df['STATION_ID_2'] = grouped_df['STATION_ID'].apply(lambda x: x[1] if len(x) > 1 else None)

# Drop the original list column to keep only the new columns
grouped_df = grouped_df.drop(columns=['STATION_ID'])

In [40]:
_df = grouped_df.copy()
_df['mode'] = _df['Ac_Mode_Model'].str[0:1].str.lower() + _df['Linked_Mode_txt']
_df.rename(columns={'Purp5_text':'purpose','PK_OK':'period'}, inplace=True)
_df['model'] = 'OBS'
obs_results_CRT_df = _df.groupby(['model','STATION_ID_1','STATION_ID_2','purpose','period','mode'],as_index=False).agg(trips_total=('linked_weight_adj', 'sum'))
obs_results_CRT_df

,model,STATION_ID_1,STATION_ID_2,purpose,period,mode,trips_total
0,OBS,01-Ogden,02-Roy,HBC,PK,dCRT,11.502
1,OBS,01-Ogden,02-Roy,HBO,PK,wCRT,21.355
2,OBS,01-Ogden,02-Roy,HBW,OK,wCRT,21.668
3,OBS,01-Ogden,02-Roy,HBW,PK,dCRT,4.021
4,OBS,01-Ogden,02-Roy,HBW,PK,wCRT,6.670
...,...,...,...,...,...,...,...
753,OBS,14-Orem Central,15-Provo Central,HBO,PK,dCRT,50.665
754,OBS,14-Orem Central,15-Provo Central,HBW,OK,dCRT,11.426
755,OBS,14-Orem Central,15-Provo Central,HBW,OK,wCRT,16.298
756,OBS,14-Orem Central,15-Provo Central,HBW,PK,wCRT,33.529


# Combine and Compare

In [41]:
results_CRT_df = pd.concat([model_results_CRT_df,obs_results_CRT_df])
results_CRT_df

,model,STATION_ID_1,STATION_ID_2,purpose,period,mode,trips_total
0,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBC,PK,dCRT,6.9294
1,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBC,PK,wCRT,7.2227
2,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,OK,dCRT,1.3716
3,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,OK,wCRT,47.8966
4,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,PK,dCRT,0.4969
...,...,...,...,...,...,...,...
753,OBS,14-Orem Central,15-Provo Central,HBO,PK,dCRT,50.6650
754,OBS,14-Orem Central,15-Provo Central,HBW,OK,dCRT,11.4260
755,OBS,14-Orem Central,15-Provo Central,HBW,OK,wCRT,16.2980
756,OBS,14-Orem Central,15-Provo Central,HBW,PK,wCRT,33.5290


In [46]:
check_df = results_CRT_df.groupby(['model','mode'], as_index=False).agg(trips_total=('trips_total','sum')).pivot(index='model',columns='mode',values='trips_total')
check_df['Total'] = check_df['dCRT'] + check_df['wCRT']
check_df

mode,dCRT,wCRT,Total
model,,,
E2.13.4.3-PurpPrd-TLF-RunFac-2.5,13426.8838,6438.1084,19864.9922
E2.13.5.2-PurpPrd-TLF-RunFac-2.5-wIvtTest,13499.0891,6449.9355,19949.0246
E2.14.4-TDM-Recalib-IvtClean-RunFac2.5,13409.0332,6442.6235,19851.6567
E2.7.3-IVT-Test,13432.8017,6484.0125,19916.8142
OBS,14571.6620,5669.1690,20240.8310


In [116]:
# remove fields that don't line up
remove_df = pd.DataFrame([
    ['03-Clearfield, 06-Woods Cross','08-Salt Lake Central, 11-Draper'],
    ['03-Clearfield, 08-Salt Lake Central','06-Woods Cross, 11-Draper'],
    ['03-Clearfield','06-Woods Cross'],
    ['08-Salt Lake Central','11-Draper']
], columns=(['STATION_ID_1','STATION_ID_2']))


# Remove matching rows using a merge with indicator
_df_filtered = results_CRT_df.merge(remove_df, on=['STATION_ID_1', 'STATION_ID_2'], how='left', indicator=True)

# Keep only rows that do not match (i.e., are not in remove_df)
_df_filtered = _df_filtered[_df_filtered['_merge'] == 'left_only'].drop(columns=['_merge'])

# Display result
display(_df_filtered)

,model,STATION_ID_1,STATION_ID_2,purpose,period,mode,trips_total
0,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBC,PK,dCRT,6.9294
1,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBC,PK,wCRT,7.2227
2,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,OK,dCRT,1.3716
3,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,OK,wCRT,47.8966
4,E2.13.4.3-PurpPrd-TLF-RunFac-2.5,01-Ogden,02-Roy,HBO,PK,dCRT,0.4969
...,...,...,...,...,...,...,...
6259,OBS,14-Orem Central,15-Provo Central,HBO,PK,dCRT,50.6650
6260,OBS,14-Orem Central,15-Provo Central,HBW,OK,dCRT,11.4260
6261,OBS,14-Orem Central,15-Provo Central,HBW,OK,wCRT,16.2980
6262,OBS,14-Orem Central,15-Provo Central,HBW,PK,wCRT,33.5290


In [121]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Widgets
purpose_options = list(_df_filtered['purpose'].unique())
period_options = list(_df_filtered['period'].unique())
mode_options = list(_df_filtered['mode'].unique())
model_options = list(_df_filtered['model'].unique())

purpose_select = widgets.SelectMultiple(options=purpose_options, value=purpose_options, description="Purpose")
period_select = widgets.SelectMultiple(options=period_options, value=period_options, description="Period")
mode_select = widgets.SelectMultiple(options=mode_options, value=mode_options, description="Mode")
model_1_dropdown = widgets.Dropdown(options=model_options, value=model_options[0], description="Model 1")
model_2_dropdown = widgets.Dropdown(options=['None'] + model_options, value="None", description="Model 2")

# Checkbox to switch between absolute and percent difference
diff_type_checkbox = widgets.Checkbox(value=False, description="Show Percent Difference")


# Function to filter _df_filtered and display pivot table
def update_table(purpose, period, mode, model_1, model_2, show_percent):

    output.clear_output()  # Clear previous output before displaying new content
    global firstTime
    if firstTime:

        filtered_data_1 = _df_filtered[
            (_df_filtered['purpose'].isin(purpose)) & 
            (_df_filtered['period'].isin(period)) & 
            (_df_filtered['mode'].isin(mode)) & 
            (_df_filtered['model']==model_1)
        ]

        filtered_data_1 = filtered_data_1.groupby(['STATION_ID_1','STATION_ID_2'],as_index=False).agg(trips_total=('trips_total','sum'))

        if model_2 != 'None': 

            filtered_data_2 = _df_filtered[
                (_df_filtered['purpose'].isin(purpose)) & 
                (_df_filtered['period'].isin(period)) & 
                (_df_filtered['mode'].isin(mode)) & 
                (_df_filtered['model']==model_2)
            ]

            filtered_data_2 = filtered_data_2.groupby(['STATION_ID_1','STATION_ID_2'],as_index=False).agg(trips_total=('trips_total','sum'))
            
            # Merge data on common keys to align before subtraction
            merged_data = pd.merge(
                filtered_data_1, 
                filtered_data_2, 
                on=['STATION_ID_1', 'STATION_ID_2'], 
                suffixes=('_m1', '_m2'), 
                how='outer'
            ).fillna(0)  # Fill missing values with 0 for subtraction

            if show_percent:
                print ("0")
            else:
                # Compute absolute difference
                merged_data['trips_total_diff'] = merged_data['trips_total_m1'].fillna(0) - merged_data['trips_total_m2'].fillna(0)
                
                # Create pivot table
                pivot = merged_data.pivot(
                    index='STATION_ID_1', 
                    columns='STATION_ID_2', 
                    values='trips_total_diff'
                ).fillna(0).astype(int).replace(0, "")  # Fill NaN with 0, then convert to int


        else:
            # Create pivot table
            pivot = filtered_data_1.pivot(
                index='STATION_ID_1', 
                columns='STATION_ID_2', 
                values='trips_total'
            ).fillna(0).astype(int).replace(0, "")   # Fill NaN with 0, then convert to int

        # Display pivot table
        display(pivot)  # Don't use display() here

    else:
        firstTime = True

# Interactivity
firstTime = False

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([purpose_select, period_select, mode_select, model_1_dropdown, model_2_dropdown, diff_type_checkbox])

# Display interactive output correctly
out = widgets.interactive_output(update_table, {
    'purpose': purpose_select,
    'period': period_select,
    'mode': mode_select,
    'model_1': model_1_dropdown,
    'model_2': model_2_dropdown,
    'show_percent': diff_type_checkbox
})

display(hbox, out)
display(output)


Output()

Output()